# 0. Settings

In [ ]:
import json
import os


ROOT_PATH = "./data"
FILE_NAME = "data_ilt.json"
MODEL_NAME = "llama-3.1-8b"
DATA_FILE_PATH = ROOT_PATH + "/" + MODEL_NAME + "/" + FILE_NAME

# 1. Functions

In [ ]:
def load_data_from_json(file_path):
    """Loads data from a JSON file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: File not found - {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file - {file_path}")
        return None

In [ ]:
def save_list_to_json(data_list, file_path):
    """Saves a list of IDs to a JSON file."""
    try:
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data_list, f, ensure_ascii=False, indent=4)
        print(f"Data successfully saved to {file_path}.")
    except Exception as e:
        print(f"Error: Failed to save {file_path}: {e}")

In [ ]:
def check_answer_consistency(data_dict):
    """
    Checks if the values in the 'answers' list for each ID are exactly identical.
    - e.g., ["A", "A", "A"] -> Consistent
    - e.g., ["A", " A", "A"] -> Inconsistent (due to whitespace)
    """
    if not data_dict:
        return None

    consistent_ids = []
    inconsistent_ids = []

    for unique_id, element_data in data_dict.items():
        answers = element_data.get("answers")

        # Skip if 'answers' key is missing or the value is not a list.
        if not isinstance(answers, list) or not answers:
            continue

        # Efficiently check if all elements in the list are identical using a set.
        # If the set length is <= 1 (list is empty, has one item, or all items are the same),
        # it's considered consistent.
        if len(set(answers)) <= 1:
            consistent_ids.append(unique_id)
        else:
            inconsistent_ids.append(unique_id)

    return {
        "consistent_ids": consistent_ids,
        "inconsistent_ids": inconsistent_ids
    }

In [ ]:
def check_normalized_answer_consistency_with_special_tokens(data_dict, special_tokens):
    """
    Checks if the values in the 'answers' list for each ID are identical
    after removing specific whitespace tokens.

    - Example tokens to remove: ' ' (U+2581), 'Ġ' (U+0120)
    - e.g., ["ĠA", " A", "A"] -> ["A", "A", "A"] after cleaning -> Consistent
    - e.g., ["A", " B"] -> ["A", "B"] after cleaning -> Inconsistent
    """
    if not data_dict:
        return None

    # List of special tokens to remove
    # Other tokens can be added to this list as needed.
    SPECIAL_TOKENS_TO_REMOVE = special_tokens

    consistent_ids = []
    inconsistent_ids = []

    for unique_id, element_data in data_dict.items():
        answers = element_data.get("answers")

        if not isinstance(answers, list) or not answers:
            continue

        # Function to remove special tokens from each answer
        def clean_answer(answer_str):
            if not isinstance(answer_str, str):
                return answer_str # Return original if not a string

            cleaned_str = answer_str
            for token in SPECIAL_TOKENS_TO_REMOVE:
                cleaned_str = cleaned_str.replace(token, "")
            return cleaned_str

        # Clean each answer using a list comprehension.
        cleaned_answers = [clean_answer(ans) for ans in answers]

        # Efficiently check if all elements in the cleaned list are identical using a set.
        if len(set(cleaned_answers)) <= 1:
            consistent_ids.append(unique_id)
        else:
            inconsistent_ids.append(unique_id)

    return {
        "consistent_ids": consistent_ids,
        "inconsistent_ids": inconsistent_ids
    }

In [ ]:
def run_analysis(data_path):
    """Runs the entire analysis process and summarizes/saves the results."""
    print(f"--- Analysis Start: {data_path} ---")

    # 1. Load data
    main_data = load_data_from_json(data_path)
    if not main_data:
        print("Analysis stopped.")
        return

    # 2. Analyze Answer Consistency
    results = check_answer_consistency(main_data)
    if not results:
        print("No data to analyze.")
        return

    # 3. Print result summary
    consistent_count = len(results["consistent_ids"])
    inconsistent_count = len(results["inconsistent_ids"])
    total_count = consistent_count + inconsistent_count

    print("\n" + "-"*50)
    print("--- Answer Consistency Analysis Results ---")
    if total_count > 0:
        print(f"Total IDs processed: {total_count}")
        print(f"  - Consistent IDs: {consistent_count} ({consistent_count/total_count:.2%})")
        print(f"  - Inconsistent IDs: {inconsistent_count} ({inconsistent_count/total_count:.2%})")
    else:
        print("No valid IDs to analyze.")
    print("-" * 50 + "\n")

    # 4. Save ID lists to files
    output_dir = os.path.dirname(data_path)
    save_list_to_json(results["consistent_ids"], os.path.join(output_dir, "consistent_ids_converted.json"))
    save_list_to_json(results["inconsistent_ids"], os.path.join(output_dir, "inconsistent_ids_converted.json"))

    print("\n--- Analysis Complete ---")

In [ ]:
def run_normalized_analysis(data_path, special_tokens):
    """
    Loads data, runs normalization-based consistency analysis,
    and summarizes/saves the results.
    """
    print(f"--- Normalization-Based Analysis Start: {data_path} ---")

    # 1. Load data
    main_data = load_data_from_json(data_path)
    if not main_data:
        print("Analysis stopped.")
        return

    # 2. Run Answer consistency analysis after normalization
    # ✅ Key Change: Calls the new function that includes normalization.
    results = check_normalized_answer_consistency_with_special_tokens(main_data, special_tokens)
    if not results:
        print("No data to analyze.")
        return

    # 3. Print result summary
    consistent_count = len(results["consistent_ids"])
    inconsistent_count = len(results["inconsistent_ids"])
    total_count = consistent_count + inconsistent_count

    print("\n" + "-"*50)
    # ✅ Changed output message: Clarifies the type of analysis.
    print("--- Answer Consistency Analysis Results (After Normalization) ---")
    if total_count > 0:
        print(f"Total IDs processed: {total_count}")
        print(f"  - ✅ Consistent IDs: {consistent_count} ({consistent_count/total_count:.2%})")
        print(f"  - ❌ Inconsistent IDs: {inconsistent_count} ({inconsistent_count/total_count:.2%})")
    else:
        print("No valid IDs to analyze.")
    print("-" * 50 + "\n")

    # 4. Save ID lists to files
    output_dir = os.path.dirname(data_path)
    # ✅ Filename Change: Specifies the analysis type in the filename.
    consistent_filename = "normalized_consistent_ids.json"
    inconsistent_filename = "normalized_inconsistent_ids.json"

    save_list_to_json(results["consistent_ids"], os.path.join(output_dir, consistent_filename))
    save_list_to_json(results["inconsistent_ids"], os.path.join(output_dir, inconsistent_filename))

    print("\n--- Analysis Complete ---")

# 2. Run

In [ ]:
DATA_FILE_PATH

'/content/drive/MyDrive/KAIRI_Experiment/Prompt_Bias/llama-3.1-8b/data_ilt_converted.json'

In [ ]:
run_analysis(DATA_FILE_PATH)

--- 분석 시작: /content/drive/MyDrive/KAIRI_Experiment/Prompt_Bias/llama-3.1-8b/data_ilt_converted.json ---

--------------------------------------------------
--- Answer 일관성 분석 결과 ---
처리된 총 ID: 977
  - Consistent IDs: 320개 (32.75%)
  - Inconsistent IDs: 657개 (67.25%)
--------------------------------------------------

데이터가 /content/drive/MyDrive/KAIRI_Experiment/Prompt_Bias/llama-3.1-8b/consistent_ids_converted.json에 성공적으로 저장되었습니다.
데이터가 /content/drive/MyDrive/KAIRI_Experiment/Prompt_Bias/llama-3.1-8b/inconsistent_ids_converted.json에 성공적으로 저장되었습니다.

--- 분석 완료 ---


# 3. Run Normalized CLS

In [ ]:
DATA_FILE_PATH

'/content/drive/MyDrive/KAIRI_Experiment/Prompt_Bias/llama-3.1-8b/data_ilt_converted.json'

In [ ]:
special_tokens = ['▁', 'Ġ']

In [ ]:
run_normalized_analysis(DATA_FILE_PATH, special_tokens)

--- 정규화 기반 분석 시작: /content/drive/MyDrive/KAIRI_Experiment/Prompt_Bias/llama-3.1-8b/data_ilt_converted.json ---

--------------------------------------------------
--- 정규화(Normalization) 후 Answer 일관성 분석 결과 ---
처리된 총 ID: 977
  - ✅ Consistent IDs: 320개 (32.75%)
  - ❌ Inconsistent IDs: 657개 (67.25%)
--------------------------------------------------

데이터가 /content/drive/MyDrive/KAIRI_Experiment/Prompt_Bias/llama-3.1-8b/normalized_consistent_ids_converted.json에 성공적으로 저장되었습니다.
데이터가 /content/drive/MyDrive/KAIRI_Experiment/Prompt_Bias/llama-3.1-8b/normalized_inconsistent_ids_converted.json에 성공적으로 저장되었습니다.

--- 분석 완료 ---
